# 7-2: Implementation of LeNet

### Code.7-2-1: LeNet with Model Sub-classing

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class LeNet(Model):
    def __init__(self):
        super(LeNet, self).__init__()

        self.conv1 = Conv2D(filters = 6, kernel_size = 5, padding = 'same',
                            activation = 'tanh')
        self.conv1_pool = AveragePooling2D(pool_size = 2, strides = 2)

        self.conv2 = Conv2D(filters = 16, kernel_size = 5, padding = 'valid',
                            activation = 'tanh')
        self.conv2_pool = AveragePooling2D(pool_size = 2, strides = 2)

        self.conv3 = Conv2D(filters = 120, kernel_size = 5, padding = 'valid',
                            activation = 'tanh')

        self.flattn = Flatten()

        self.dense1 = Dense(units = 84, activation = 'tanh')
        self.dense2 = Dense(units = 10, activation = 'softmax')

    def __call__(self, x):
        print("x: {}".format(x.shape))

        x = self.conv1(x)
        print("x: {}".format(x.shape))
        x = self.conv1_pool(x)
        print("x: {}".format(x.shape))

        x = self.conv2(x)
        print("x: {}".format(x.shape))
        x = self.conv2_pool(x)
        print("x: {}".format(x.shape))

        x = self.conv3(x)
        print("x: {}".format(x.shape))
        x = self.flattn(x)
        print("x: {}".format(x.shape))

        x = self.dense1(x)
        print("x: {}".format(x.shape))
        x = self.dense2(x)
        print("x: {}".format(x.shape))

        return x 
    
model = LeNet()
x = tf.random.normal(shape = (32, 28, 28, 1))
predictions = model(x)
print(predictions.shape)


Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

x: (32, 28, 28, 1)
x: (32, 28, 28, 6)
x: (32, 14, 14, 6)
x: (32, 10, 10, 16)
x: (32, 5, 5, 16)
x: (32, 1, 1, 120)
x: (32, 120)
x: (32, 84)
x: (32, 10)
(32, 10)


### Code.7-2-2: LeNet with Hybrid Method

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

from tensorflow.keras.datasets import mnist
from tensorflow.keras.losses import SparseCategoricalCrossentropy


#### LeNet Implementation ####
class ConvLayer(Layer):
    def __init__(self, filters, padding, pool=True):
        super(ConvLayer, self).__init__()
        self.pool = pool
        self.filters = filters
        self.padding = padding
        self.conv = Conv2D(filters = self.filters, kernel_size = 5, padding = self.padding,
                           activation = 'tanh')
        if self.pool == True:
            self.conv_pool = AveragePooling2D(pool_size = 2, strides = 2)

    def __call__(self, x):
        x = self.conv(x)
        
        if self.pool == True:
            x = self.conv_pool(x)

        return x
    

class LeNet(Model):
    def __init__(self):
        super(LeNet, self).__init__()   

        self.conv1 = ConvLayer(filters = 6, padding = 'same')
        self.conv2 = ConvLayer(filters = 16, padding = 'valid')
        self.conv3 = ConvLayer(filters = 120, padding = 'valid', pool = False)
        self.flatten = Flatten()

        self.dense1 = Dense(units = 84, activation = 'tanh')
        self.dense2 = Dense(units = 10, activation = 'softmax')

    def __call__(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x) 

        x = self.dense1(x)
        x = self.dense2(x)

        return x


##### Dataset Preparation #####
(train_images, train_labels), _ = mnist.load_data()
train_images = np.expand_dims(train_images, axis = 3).astype(np.float32)
trian_labels = train_labels.astype(np.int32)
# print(train_images.shape, train_images.dtype)
# print(train_labels.shape, train_labels.dtype)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.batch(32)


#### Forward Propagation ####
model = LeNet()
loss_object = SparseCategoricalCrossentropy()

for images, labels in train_dataset:
    # print(images.shape, labels.shape)
    predictions = model(images)
    loss = loss_object(labels, predictions)
    print(loss)
    break





6
16
120
tf.Tensor(2.3548458, shape=(), dtype=float32)


### Code.7-2-3: Forward Propagation of LeNet